# 0. [OPTIONAL] Installing course dependencies

These are dependencies for the whole course.

In [71]:
!pip install -r ../requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


You may skip the next block for now. You will need `ffmpeg` on week 12.

In [72]:
# !conda update -y base conda
!conda install -c conda-forge ffmpeg -y

fish: Unknown command: conda
fish: 
conda install -c conda-forge ffmpeg -y
^~~~^


Run the next cell if you want to download embedding model, but this is not required during this lab. You can do it later.

In [73]:
!python -m spacy download en_trf_distilbertbaseuncased_lg

/Users/dartt0n/.rye/py/cpython@3.12.3/bin/python3: No module named spacy


# 1. Touching the Internet

Solve the following task.
1. Download [this page](https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt)
2. Save it to the file with the **unique** name derived from the URL. NB File with another URL should not be save into the file with this name. E.g. [this file](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/facts.txt) is another file with another content!

Hints:
- [requests](https://docs.python-requests.org/en/latest/) library is cool.
- [hashlib](https://docs.python.org/3/library/hashlib.html) may help with computing hash strings.
- when you download and save the data, don't try to encode and decode it. Use binary format when working with streams and files. <span style="color:red">Discuss with your TA which encodings you know and how they differ</span>.

In [74]:
import requests
from hashlib import sha256

url1 = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
url2 = "https://github.com/IUCVLab/information-retrieval/blob/main/datasets/facts.txt"


def save_file(url, extenstion: str = "") -> str:
    filename = sha256(url.encode()).hexdigest() + extenstion
    response = requests.get(url=url)
    response.raise_for_status()

    with open(filename, "wb") as f:
        f.write(response.content)

    return filename


print("file 1: ", save_file(url1, ".txt"))
print("file 2: ", save_file(url2, ".txt"))

file 1:  e2d9cd700ca8f8c01cd68193a1249d29030a1782108055b444a5ead7ab310425.txt
file 2:  7770f104796093e196fac1e6822438ba9a2dadc33030edb373caf84a4fc99005.txt


# 2. Parsing different formats

Most probably, if you meet something in the Internet, this is one of: binary, plain text, XML, or json. XML then splits into xHTML, RSS, Atom, SOAP, XML-RPC, ... . Your task is to learn, how to process different formats.

## 2.1. JSON

In [the given file](http://sprotasov.ru/data/postnauka.txt) there is valid json. Parse this file and print all video URLs, which have `computer science` tag. Use built-in features of `requests`, or just a `json` library ([ref](https://docs.python.org/3/library/json.html)).

Hint:
- if the file has issues with parsing read about [the difference](https://stackoverflow.com/questions/57152985/what-is-the-difference-between-utf-8-and-utf-8-sig).

In [75]:
import json
import requests
import io

url = "http://sprotasov.ru/data/postnauka.txt"

# requests.get(url).json()
# data = json.load(io.BytesIO(requests.get(url).content))  # buffered for memory optimization
data = json.loads(requests.get(url).content)
for entry in data:
    if "computer science" in entry["tags"]:
        print(entry)

{'title': 'Компьютерные науки: от программирования до искусственного интеллекта', 'id': '0004', 'parentId': None, 'tags': ['computer science', 'interview', 'radio', 'postnauka'], 'author': 'Stanislav Protasov', 'url': 'http://postnauka.ru/talks/31897', 'description': 'Интервью об особенностях компьютерных наук, параллельном программировании и искусственном интеллекте', 'contentType': 'article', 'image': 'http://cdn.postnauka.netdna-cdn.com/img/2014/09/Computer-Science.jpg'}
{'title': 'Внутренняя разработка в Computer Science', 'id': '0005', 'parentId': None, 'tags': ['computer science', 'internal development', 'video', 'postnauka'], 'author': 'Stanislav Protasov', 'url': 'http://postnauka.ru/video/24306', 'description': 'О математических задачах в компьютерной науке, обработке отчетов об ошибках и тестировании ПО', 'contentType': 'video', 'image': 'http://cdn.postnauka.netdna-cdn.com/img/2014/04/Stanislav-Protasov-Vnutrennyaya-razrabotka-v-omputer-Science.jpg'}
{'title': 'Что такое Big

## 2.2. HTML

For a given StackExchange answer extract logins of the contributors (who asked and who answered) with votes. [bs4](https://beautiful-soup-4.readthedocs.io/en/latest/) will help you to do the job.

I can recommend to use CSS or XPath selectors. `div` elements with `post-layout` class represent answers. Inside there are `div` with `votecell` class stroring votes number and `div` with class `user-details` storing user info. My personal recommendation is to use `css selectors`, which are [documented here](https://beautiful-soup-4.readthedocs.io/en/latest/#css-selectors).

In [76]:
import requests
from bs4 import BeautifulSoup

url = (
    "https://math.stackexchange.com/questions/411486/"
    "understanding-the-singular-value-decomposition-svd"
)
print(url)

# TODO. Your code here should parse HTML source page and find contributors of the repository.
html = BeautifulSoup(requests.get(url).content)

pprint([x.text for x in html.select(".user-details > a")])

https://math.stackexchange.com/questions/411486/understanding-the-singular-value-decomposition-svd
['Rodrigo de Azevedo',
 'Celdor',
 'Ittay Weiss',
 'Tomasz Bartkowiak',
 'Bart Vanderbeke',
 'Bart Vanderbeke',
 'littleO',
 'hgfei',
 'TheSHETTY-Paradise']


# 2.3. RSS feed

A lot of information is already organized in typed XML documents. Podcasts, for example, are just RSS feed. Parse [the feed of this podcast](http://sprotasov.ru/podcast/rss.xml) and print out:
- the number of episodes
- the length of the time span between the first and the last episodes (in days).

Use [`feedparser` library for this](https://waylonwalker.com/parsing-rss-python/).

In [77]:
from datetime import datetime
from time import mktime
import feedparser
import jq
from pprint import pprint

rss = "http://sprotasov.ru/podcast/rss.xml"

# TODO: complete the code to compute the time span of all the episodes.
last, *_, first = map(lambda x: x["published_parsed"], feedparser.parse(rss)["entries"])

last = datetime.fromtimestamp(mktime(last))
first = datetime.fromtimestamp(mktime(first))

print(last - first)

2030 days, 9:25:00


# 3. [EXTRA TASK] Solving simple information retrieval task

According to the name, `information retrieval` is the discipline, which helps retrieves information (from unstructured sources). Thus, we will retrieve some information from [this news article](https://www.bbc.com/news/world-us-canada-59944889). Your task is to write a code, which will answer the question: **How many people die every day in the US waiting for a transplant?** Write flexible enough code. Test yourself by changing the link to [this one](https://www.americantransplantfoundation.org/about-transplant/facts-and-myths/).

In [78]:
import nltk

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/dartt0n/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [102]:
import requests
from nltk import sent_tokenize
from bs4 import BeautifulSoup
from pprint import pprint
from Levenshtein import distance as levenshtein_distance


url = "https://www.bbc.com/news/world-us-canada-59944889"
url2 = "https://www.americantransplantfoundation.org/about-transplant/facts-and-myths/"

question = "How many people die every day in the US waiting for a transplant?"


html = BeautifulSoup(
    requests.get(
        url2,
        headers={
            # fake user agent
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15"
        },
    ).content
)

text = html.text

while '\n\n' in text:
    text = text.replace('\n\n', '\n')

text = text.strip()

In [103]:
dataset_documents = sent_tokenize(text)

In [104]:
from collections import defaultdict

sentences = [x.split() for x in dataset_documents]
terms = question.split()

term_freq = defaultdict(lambda: 0.0)

for index, sentence in enumerate(sentences):
    for term in terms:
        if term in sentence:
            term_freq[index] += 1

    term_freq[index] /= len(sentence)

for rank, (index, _) in enumerate(sorted(term_freq.items(), key=lambda x: x[1], reverse=True)[:10]):
    print(f"- {rank+1}. {dataset_documents[index]}")

- 1. On average, 16 people die every day from the lack of available organs for transplant.
- 2. Over 104,000 people in the United States are currently on the waiting list for a lifesaving organ transplant.
- 3. Over 850,000 transplants have occurred in the U.S. since 1988.
- 4. There are nearly as many people dying per year of organ disease as are on the transplant waiting list currently!
- 5. The US saw a 7.5% increase in deceased donors last year- from 13,863 in 2021 to 14,903 in 2022
Another name is added to the national transplant waiting list every 9 minutes.
- 6. If you are rich or a celebrity, you can move up the waiting list more quickly.
- 7. One in three donors are not biologically related to the recipient.
- 8. More than 44,000 corneal transplants take place each year in the United States.
- 9. The first priority of a medical professional is to save lives when sick or injured people come to the hospital.
- 10. After donating an organ or tissue, a closed casket funeral is the